# Separation prediction

In this section we will combine the vortex panel method and a boundary layer solver to predict separation on any 2D shape.


---
<img src="resources/graphics4.png" width="400">

Recall that the Pohlhausen profile is used to describe a **laminar** velocity profile exposed to external pressure gradients. The profile is defined as

$$ \frac u {u_e} = f(\eta,\lambda) = P_F(\eta)+\lambda P_G(\eta) $$

where $\lambda$ is the *shape factor*, given by
$$ \lambda = \frac {\delta^2}\nu u_e'$$

and the profile shapes are defined by

- $P_F = 2\eta-2\eta^3+\eta^4$ is the flat plate profile
- $P_G = \frac\eta 6 (1-\eta)^3$ is the modification for pressure gradients

These can be easly defined using a set of python functions


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

def pohlF(eta): return 2*eta-2*eta**3+eta**4
def pohlG(eta): return eta/6*(1-eta)**3

def pohlPlot(lam):
    plt.figure(figsize=(5,5))
    plt.xlabel(r'$u/u_e$', fontsize=24)
    plt.ylabel(r'$y/\delta$', fontsize=24)
    eta = np.linspace(0.0,1.0)
    plt.plot(pohlF(eta),eta, ls='--', label=r'$f(0)$')
    plt.plot(pohlF(eta)+lam*pohlG(eta),eta, lw=2, label=r'$f(\lambda)$')
    plt.legend(loc='upper left', fontsize=16)

In [ ]:
pohlPlot(lam=0)

## Thwaites solution method

Thwaites was able to substitue the Pohlhausen profile and integrate the momentum equation into the form 

$$ \delta^2_2(x) =\frac{0.45\nu}{u^6_e(x)}\int^x_0 u_e^5(x) dx $$

and we would like to get this into a non-dimensional form. To do so, we define $s=x/L$ and $u_s = u_e/U$, where $U$ and $L$ are characteristic values for the problem. Substitution gives

$$ \frac{\delta_2}{L} \sqrt{Re_L} =\left[\frac{0.45}{u^6_s(s)}\int^s_0 u_s^5(s) ds \right]^{1/2}$$


The right hand side can be implemented very easily in python. We simply integrate over a velocity array `u_s` of length `N` with `s` as the corresponding distance along the boundary layer:

```python
numpy.sqrt(0.45/u_s**6*cumtrap(u_s**5*,s,initial=0))
```

where [`cumtrapz`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.cumtrapz.html) is the cumulative version of trapz. We can scale the momentum thickness later as needed.

##### Quiz

Assuming the same array size $N$, how long will it take to solve boundary layer equation compared to the vortex panel equation?

1. Boundary layer will take **less** time
1. Boundary layer layer will take **the same** amount of time
1. Boundary layer will take **more** time

---

This has been implemented in the `thwaites` function in the `BoundaryLayer` module. Lets import the module (as `bl`) and validate the function on a flat plate.

In [ ]:
import VortexPanel as vp
import BoundaryLayer as bl

N=32                             # number of steps
s = np.linspace(0,1,N)           # scaled distance
u_s = np.ones_like(s)            # flat plate (uniform) external velocity
delta2,_,_ = bl.thwaites(s,u_s)  # thwaites momentum thickness

plt.plot(s,0.664*np.sqrt(s),'r',label='Blasius')
plt.plot(s,0.685*np.sqrt(s),'k',label='Pohlhausen')
plt.plot(s,delta2,'o',label='Thwaites')
plt.ylabel(r'$\frac{\delta_2}{L} \sqrt{Re_L}$', fontsize=16,rotation=0,y=1)
plt.xlabel(r'$x/L$', fontsize=16)
plt.legend(loc='lower right');

As we saw before, Pohlhausen and Blausis are only 3% different on a flat plate, and Thwaites tuned his result to be even closer to the exact result.

## Profile shape and separation

Separation is our primary concern, not the momentum thickness. Algebra gives

$$ \frac{du_s}{ds}\frac{\delta_2^2}{L^2}Re_L = \lambda c_2^2 = f(\lambda) $$

where the left hand side is known from Thwaites and the prescribed $u_s$ array, and $c_2=\delta_2/\delta$ which is a known function of $\lambda$. Therefore, we can determine $\lambda$ on every point along the body.


##### Quiz

What value of $\lambda$ denotes separation?

1. $\lambda$=-24
1. $\lambda$=-12
1. $\lambda$=0
1. $\lambda$=12

----

Lets take a look at the `bl.thwaites` function help:

In [ ]:
help(bl.thwaites)

Notice that this function doesn't only return the momentum thickness, but also the shape factor $\lambda$ and the array index of separation `iSep` found using the equations above. 

The help example code uses the external velocity over a unit circle. This is a good test case because the _theoretical_ solution is known to be $108\deg=1.88\text{rad}$ from the leading stagnation point.


![short](resources/graphics3.png)


##### Numerical fundamental: Validation
##### Every piece of code must be tested against a known nontrivial example

---

So lets validate $\lambda$ and `iSep`.

In [ ]:
s = np.linspace(0,np.pi,32)          # distance along BL
u_s = 2*np.sin(s)                    # circle external velocity
delta2,lam,iSep = bl.thwaites(s,u_s) # BL properties

fig,ax=plt.subplots(2,1,sharex=True)
ax[0].plot(s,delta2,'o-')
ax[0].set_ylabel(r'$\delta_2 \sqrt{\frac{U}{\nu R}}$',rotation=0,y=.9,size=16)
ax[1].plot(s,lam,'o-')
ax[1].axhline(-12,ls='--')
ax[1].set_yticks([-12,0,12])
ax[1].set_ylabel(r'$\lambda}$',rotation=0,size=16)
plt.xticks([0,np.pi/2,np.pi],['0','$\pi/2$','$\pi$'])
plt.xlabel(r'$\theta$',size=16)
plt.show()

Near the stagnation point, the pressure gradient is favourable and very strong, limiting the BL growth. As you move along the boundary layer, $\lambda$ decreases and eventually crosses the threshold for separation $\lambda=-12$. Values after separation are meaningless and have been zeroed out in the function.

From the plot we can see separation occurs soon after $\pi/2$, but this is too vague. We need a specific prediction and we can use `iSep` to get one. If `iSep=13.2` it means that separation occured 20% of the way between `s[13]` and `s[14]`. All we need to do is interpolate, and we can do this for __any__ panel attribute using the `BoundaryLayer.sep` function. Let's see what we get:

In [ ]:
sSep = bl.sep(s,iSep)
print('analytic prediction @ s=1.88 \n numeric prediction @ s={:.2f}'.format(sSep))

Our numerical method prediction is __one percent different__ than the theoretical result!

##### Quiz

How does the separation point depend on $\nu$?

1. Increasing $\nu$ delays separation
1. Decreasing $\nu$ delays separation
1. Chaning $\nu$ has no effect on separation

## Panel method and boundary layer coupling

This works for a circle, but what about something more complicated, like a foil at an angle of attack? We can use the `VortexPanel` module!

What will we need to interface the `VortexPanel` and `BoundaryLayer` modules?  The external flow doesn't need anything from `BoundaryLayer` - it just needs a geometry and angle of attack:

In [ ]:
import VortexPanel as vp
alpha = np.pi/20
foil = vp.make_jfoil(N=64)
foil.solve_gamma_O2(alpha,kutta=[(0,-1)])
foil.plot_flow()

---

Thwaites doesn't need the details of the geometry, but it does need:

- $s$: the scaled distance along the boundary layer
- $u_s$: the scaled velocity on the edge of the boundary layer


These are suprising simple. Since $|U|=1$, the 
We know 

$$u_s(x_i,y_i) = -\gamma_i$$

and the external velocity is $u_e=|u_s|$.

---

Now that we know how to get the velocity, we need to think about the distance along the BL, $s$. Note that a body will form **two** boundary layers, one on each side. We need to identify the starting point of these two flow regions.

##### Quiz

Where is the starting point of the two boundary layers?

1. The first panel: `foil[0]`
1. The panel where $u_s = 0$
1. The left-most panel, `foil[N/2]`


This makes it straightforward to split the body into the two boundary layer sections:
```python
def split(self):
    top = [p for p in self.panels if p.gamma<=0]
    bot = [p for p in self.panels if p.gamma>=0]
    return PanelArray(top),PanelArray(bot[::-1])
```
Note that we changed the direction of the bottom array so that it runs from the stagnation point to the trailing edge to match the flow direction.

This is implemented in the `PanelArray.split` function. Let's use the example code to test the `split` function:

In [ ]:
foil = vp.make_jukowski(N=64)        #1. Define the geometry
foil.solve_gamma_kutta(alpha=0.1)    #2. Solve for the potential flow
foil_top,foil_bot = foil.split()     #3. Split the boundary layers

def plot_segment(segment):
    pyplot.figure(figsize=(10,2))
    pyplot.axis([-1.2,1.2,-.3,.3])
    for i,p_i in enumerate(segment.panels): 
        p_i.plot()
        if i%8 == 0:
            pyplot.scatter(p_i.xc,p_i.yc)
            pyplot.text(p_i.xc,p_i.yc+0.05, 
                'panel ['+'%i'%i+']',fontsize=12)
    pyplot.show()

plot_segment(foil_top)
plot_segment(foil_bot)

As we change `alpha`, the stagnation point - and therefore the panels in each section - changes.

---

Once the sections are `split` we can feed them to `PanelArray.march`. Again, only three lines of code:
```python
def march(self, nu, thwaites=False):
    s = self.distance()                      # distance
    u_e = abs(self.get_array("gamma"))       # velocity
    return bl.march(s, u_e, nu, thwaites)       # march
```

In [ ]:
help(foil.march)

## Boundary layer around a circle

Lets run the example in the docstring as our first application: the boundary layer on a circle.

##### Quiz

Why do I keep testing code on simple shapes like circles?

1. I'm terribly forgetful
1. New examples take work
1. I want to validate the new code

In [ ]:
# set up and solve circle BL flow:
nu = 1e-5
circle = vp.make_circle(N=64)     #1. make the geometry
circle.solve_gamma()              #2. solve the pflow
top,bottom = circle.split()       #3. split the panels
delta2,lam,iSep = top.march(nu)   #4. march along the BL

# plot lambda(s)
s = top.distance()
pyplot.ylabel(r'$\lambda$', fontsize=20, rotation=0)
pyplot.xlabel(r'$s$', fontsize=20)
pyplot.plot(s,lam,lw=2)
pyplot.axhline(-12,ls='--')

---

Is this a bug??

We know from lectures that $\delta$ scales as $\sqrt\nu$, and therefore $\lambda=\frac{\delta^2}\nu u_e'$ doesn't depend on $\nu$ at all. Since the separation point is determined by $\lambda$, this is also independent of $\nu$.

##### Fluids fundamental: Separation Point
##### The point of laminar separation is independent of $Re$

This is **not** true of a turbulent boundary layer.

## Separation of the foil

Finally, let's solve for the boundary layer separation points on the foil using the `PanelArray.sep_point` function. It is another 3 liner:
```python
def sep_point(self):
    delta2,lam,iSep = self.march(1)
    x,y = self.get_array('xc','yc')
    return bl.sep(x,iSep),bl.sep(y,iSep)
```
 Note this function doesn't depend on $\nu$ because... Quiz 3

In [ ]:
alpha = numpy.pi/18
foil = vp.make_jukowski(N=64)       #1. make the geometry
foil.solve_gamma_kutta(alpha)       #2. solve the pflow
top,bot = foil.split()              #3. split the panels
x_top,y_top = top.sep_point()       #4. find separation on top 
x_bot,y_bot = bot.sep_point()       #   ... and bottom BL

foil.plot_flow()
pyplot.scatter(x_top,y_top, s=100, c='r', zorder=10)
pyplot.scatter(x_bot,y_bot, s=100, c='g', zorder=10)
pyplot.show()

And we see that at an angle of attack, the high pressure side of the foil (the bottom) has delayed separation, while the low pressure side has early separation. This is exactly the kind of information we need to predict stall.

---

## Summary 

Let's review the general solution methodology again: 

1. `make` the geometry as a set of vortex panels
1. `solve` the potential flow by applying the no-slip and kutta conditions
1. `split` the panels into sections on either side of the stagnation points
1. `march` the boundary layer equations to the point of separation

Then you can measure, print, or plot any flow quantity you want. 

##### Numerical fundamental: Synergy
##### Simple functions can be linked to build powerful methods

By combining the `VortexPanel` methods with the `BoundaryLayer.march` method, we've developed a numerical tool to predict separation on **any** body (or group of bodies).

## Aspect ratio validation

We've validated the code for the known case of a circle, but this is just one geometry. 

##### Numerical fundamental: Skepticism
##### "That is what the computer said..." - Famous last words

We should validate a new method on as many known results as possible. 

One thing we could validate is that the separation points change properly if the aspect ratio $t/c$ is changed.  Here is a summary figure from Chapter 3 of Hoerner's *Fluid-Dynamic Drag*

---
![Hoerner Drag, Fig 5, Chap 3](resources/separation_hoerner.png)

---
There are four ellipse examples and two Jukowski examples. Based on this figure, I estimate:

Jukowski foil:

$t/c$| 0.17  | 0.15 
---|
$x/c$| $0.39$  | $0.49$ 

Ellipse:

$t/c$| 1 |1/2 | 1/4 | 1/6 | 1/8 
---|
$x/c$| $0.65$ | $0.75$ | $0.85$ | $0.89$ | $0.92$ 

where $x$ is the linear distance from the nose to the point of separation. I've included the result for a circle.

##### Your turn #5

Validate the separation prediction method against these two families of geometries. 

 - **Predict** separation for the Jukowski cases, being sure to match $t/c$.
 - **Predict and Plot** $x/c$ for the ellipse for $t/c$=`linspace(0.08,1.1,10)` compared to Hoerner's data. 
 - **Discuss** the results with regards to validation and trends. Are these adequate to validate the numerical method? Are there any surprises?

I recommend that you write a few functions to help automate the prediction process, similar to the previous notebooks...

##### Solution #5


In [ ]:
# function definitions

# x/c for the foils using vp.make_jukowski

# x/c for the ellipses using vp.make_ellipse
t_c = numpy.linspace(0.08,1.1,10)

pyplot.scatter([1,1./2.,1./4.,1./6.,1./8.],
               [0.65,0.75,0.85,0.89,0.92], 
               s=100, label='Hoerner')
pyplot.legend(loc='upper right')
pyplot.xlabel(r'$t/c$', fontsize=16)
pyplot.ylabel(r'$x/c$', fontsize=16)

Using the Pohlhausen profile, the factors in the momentum integral equation can be defined as functions of $\lambda$

- $c_1 = \frac{\delta_1}\delta = \int_0^1 (1-f) d\eta = \frac3{10}-\lambda\frac1{120}$


- $c_2=\frac{\delta_2}\delta = \int_0^1 f(1-f) d\eta = \frac{37}{315}-\lambda\frac1{945}-\lambda^2\frac1{9072}$


- $\frac 12 c_f Re_\delta =f'_0= 2+\lambda\frac1{6}$

where $Re_\delta = \frac{u_e\delta}\nu$ is the local boundary layer Reynolds number. 

I've implemented these in the `BoundaryLayer` module (which I'll nickname `bl`) as `disp_ratio`, `mom_ratio`, and `df_0`:

In [ ]:
import BoundaryLayer as bl

pyplot.figure(figsize=(5,5))
pyplot.xlabel(r'$\lambda$', fontsize=20)
lam = numpy.linspace(-12,12)
pyplot.plot(lam,bl.disp_ratio(lam), lw=2, label=r'$c_1=\delta_1/\delta$')
pyplot.plot(lam,bl.mom_ratio(lam), lw=2, label=r'$c_2=\delta_2/\delta$')
pyplot.ylim(0,)
pyplot.legend(loc='upper right',fontsize=16)